<a href="https://colab.research.google.com/github/graves/awful_text_news/blob/main/Awful_News_Llama_A100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title # Pre-setup

# Drive, for saving/loading cloudflared configs:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
#@title # Setup

# Get llama.cpp
%cd /content/drive/MyDrive
!rm -rf llama.cpp-a100
!git clone https://github.com/ggerganov/llama.cpp.git llama.cpp-a100
%cd llama.cpp-a100

# Build Llama.cpp specific to T4
!CMAKE_ARGS="-DGGML_CUDA=ON"  cmake -B build -DGGML_CUDA=ON
!cmake --build build --config Release

%cd /content/drive/MyDrive/llama.cpp-a100/build/bin

# Get cloudflared
!wget -q -nc https://github.com/cloudflare/cloudflared/releases/download/2025.4.2/cloudflared-fips-linux-amd64
!mv cloudflared-fips-linux-amd64 cloudflared
!chmod +x cloudflared

In [ ]:
#@title #Get Model

%cd /content/drive/MyDrive

# Get qwen2.5-3B-instruct
!wget https://huggingface.co/bartowski/Qwen2.5-3B-Instruct-GGUF/resolve/main/Qwen2.5-3B-Instruct-Q8_0.gguf

In [ ]:
#@title # On the First Run
# For first-time runs:
!./cloudflared tunnel login
!cp /root/.cloudflared/cert.pem /content/drive/MyDrive/cloudflared-cert.pem

In [ ]:
#@title # On Later Runs
# For later runs:
!mkdir -p /root/.cloudflared/
!cp /content/drive/MyDrive/cloudflared-cert.pem /root/.cloudflared/cert.pem

In [ ]:
#@title # Inference
# To tunnel while running a Python app:
%cd /content/drive/MyDrive/llama.cpp-a100/build/bin

!chmod +x cloudflared
!chmod +x llama-server

import os

import subprocess
import threading
import time
import socket

from subprocess import Popen

print("start cloudflared tunnel")
f = open("stdout", "w")
Popen(['./cloudflared','--url','http://localhost:12345', '--hostname', "llama.awfulsec.com"], bufsize=0, stdout=f, stderr=subprocess.STDOUT)

import time

time.sleep(10)

!grep -F trycloudflare stdout

!./llama-server -m ./Qwen2.5-3B-Instruct-Q8_0.gguf -ngl 37 -c 0 --port 12345
